<a href="https://colab.research.google.com/github/arinadi/Transcript-AI/blob/main/Transcript_AI_Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Transcript_AI_Web (Simplified Stop & Delete Runtime)
# 📌 Installing Required Libraries
!pip install -q openai-whisper ffmpeg pydub flask pyngrok

# 📌 Importing Libraries
import whisper
import os
import threading
import time

# [NEW] Import 'runtime' for self-destruction
from google.colab import runtime
from google.colab import userdata

from flask import Flask, request, render_template_string, send_from_directory, url_for, jsonify, make_response
from werkzeug.utils import secure_filename
from pyngrok import ngrok, conf

# ------------------------------------------------------------------------------
# NGROK CONFIGURATION WITH AUTHTOKEN (IMPORTANT!)
# ------------------------------------------------------------------------------
NGROK_AUTH_TOKEN = userdata.get('MyNGROK')

if NGROK_AUTH_TOKEN is None or "YOUR_NGROK_AUTH_TOKEN" in NGROK_AUTH_TOKEN:
    print("⚠️ PERINGATAN: Token NGROK tidak ditemukan. Silakan atur di Colab Secrets dengan kunci 'MyNGROK'.")
    NGROK_AUTH_TOKEN = None
else:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
    print("✅ Konfigurasi token Ngrok berhasil.")

# ------------------------------------------------------------------------------
# Core Transcription Function Section
# ------------------------------------------------------------------------------
def format_transcription_with_pauses(result, pause_threshold=0.7):
    formatted_text = ""
    previous_end = 0
    for segment in result["segments"]:
        start = segment["start"]
        text = segment["text"].strip()
        if start - previous_end > pause_threshold:
            formatted_text += "\n\n"
        formatted_text += text + " "
        previous_end = segment["end"]
    return formatted_text.strip()

def transcribe_audio_web(audio_path, model, output_folder):
    print(f"📢 Menjalankan transkripsi untuk: {audio_path}...")
    result = model.transcribe(audio_path, language=None, word_timestamps=True) # Auto-detect language
    formatted_text = format_transcription_with_pauses(result)
    base_filename = os.path.splitext(os.path.basename(audio_path))[0]
    output_filename = f"transcription_final_{secure_filename(base_filename)}.txt"
    output_filepath = os.path.join(output_folder, output_filename)
    with open(output_filepath, "w", encoding="utf-8") as f:
        f.write(formatted_text)
    print(f"✅ Transkripsi selesai! Disimpan di '{output_filepath}'.")
    return output_filename

# ------------------------------------------------------------------------------
# Flask Configuration
# ------------------------------------------------------------------------------
UPLOAD_FOLDER = 'uploads'
TRANSCRIPT_FOLDER = 'transcripts'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(TRANSCRIPT_FOLDER, exist_ok=True)

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['TRANSCRIPT_FOLDER'] = TRANSCRIPT_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 100 * 1024 * 1024 # 100MB

# 📌 Load Whisper model
model_size = 'large-v2' # @param ["medium", "large-v2"]
print(f"⏳ Memuat model Whisper '{model_size}'...")
try:
    model = whisper.load_model(model_size)
    print("✅ Model Whisper berhasil dimuat.")
except Exception as e:
    print(f"❌ Gagal memuat model Whisper: {e}")
    model = None

# Global variables
public_url_ngrok = None

# ------------------------------------------------------------------------------
# HTML Template - JavaScript 'stopService' function updated
# ------------------------------------------------------------------------------
HTML_TEMPLATE = """
<!doctype html>
<html lang="id">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Transkripsi Audio AI</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif; margin: 20px; background-color: #f4f7f6; color: #333; }
        .container { max-width: 800px; margin: auto; background-color: #fff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.1); }
        h1, h2 { color: #2c3e50; border-bottom: 2px solid #e0e0e0; padding-bottom: 10px; }
        input[type="file"] { margin-bottom: 15px; }
        input[type="button"], button { background-color: #3498db; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer; margin-right: 5px; font-size: 1em; transition: background-color 0.3s; }
        input[type="button"]:hover, button:hover { background-color: #2980b9; }
        input[type="button"]:disabled { background-color: #bdc3c7; cursor: not-allowed; }
        button.stop-button { background-color: #e74c3c; }
        button.stop-button:hover { background-color: #c0392b; }
        .result { margin-top: 25px; padding: 15px; background-color: #ecf0f1; border-radius: 5px; }
        a { color: #3498db; text-decoration: none; font-weight: bold; }
        a:hover { text-decoration: underline; }
        .loader-container { text-align: center; margin-top: 20px;}
        .loader { border: 5px solid #f3f3f3; border-top: 5px solid #3498db; border-radius: 50%; width: 30px; height: 30px; animation: spin 1s linear infinite; display: none; margin: 10px auto; }
        #statusContainer ul, #historyContainer ul { list-style-type: none; padding-left: 0; }
        #historyContainer li { margin-bottom: 8px; padding: 12px; border-radius: 4px; border-left: 5px solid #2ecc71; background-color: #e8f8f5; display: flex; justify-content: space-between; align-items: center; }
        .file-details { flex-grow: 1; }
        .file-actions a { margin-left: 10px; }
        #serviceControlArea { text-align: center; margin-top: 30px; padding-top: 20px; border-top: 1px solid #eee; }
        #serviceStatus { margin-bottom: 10px; padding: 10px; border-radius: 4px; }
        #serviceStatus.running { background-color: #d4edda; color: #155724; }
        #serviceStatus.stopped { background-color: #f8d7da; color: #721c24; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Upload Audio untuk Transkripsi</h1>
        <p>Pilih satu atau lebih file audio. Bahasa akan dideteksi secara otomatis.</p>
        <form id="uploadForm">
            <input type="file" id="audioFilesInput" name="audio_file_input" multiple required>
            <br>
            <input type="button" value="Mulai Transkripsi" onclick="startProcessingQueue()">
        </form>

        <div id="loader" class="loader"></div>

        <div id="historyContainer" class="result">
            <h2>Riwayat Transkripsi (Terbaru di Atas):</h2>
            <ul id="historyList">
                {% if initial_processed_files %}
                    {% for file_info in initial_processed_files %}
                        <li>
                            <span class="file-details"><strong>{{ file_info.original_name }}</strong></span>
                            <span class="file-actions">
                                <a href="{{ url_for('view_transcript', filename=file_info.transcript_file) }}" target="_blank">Lihat</a> |
                                <a href="{{ url_for('download_transcript', filename=file_info.transcript_file) }}">Download</a>
                            </span>
                        </li>
                    {% endfor %}
                {% else %}
                    <li id="noHistoryMsg" style="border: none; background: none;">Belum ada riwayat.</li>
                {% endif %}
            </ul>
        </div>

        <div id="serviceControlArea">
             <div id="serviceStatus">Layanan sedang dimuat...</div>
             <button type="button" class="stop-button" onclick="stopService()">Hentikan & Hapus Runtime</button>
        </div>
    </div>

    <script>
        let fileProcessingQueue = [];

        function updateServiceStatus(isRunning, message = "") {
            const statusDiv = document.getElementById('serviceStatus');
            const stopButton = document.querySelector('button.stop-button');
            const uploadButton = document.querySelector('input[value="Mulai Transkripsi"]');

            if (isRunning) {
                statusDiv.className = 'running';
                statusDiv.textContent = message || 'Layanan berjalan normal.';
                uploadButton.disabled = false;
                stopButton.disabled = false;
            } else {
                statusDiv.className = 'stopped';
                statusDiv.textContent = message;
                uploadButton.disabled = true;
                stopButton.disabled = true;
            }
        }

        async function startProcessingQueue() {
            const fileInput = document.getElementById('audioFilesInput');
            if (fileInput.files.length === 0) { alert("Silakan pilih file audio terlebih dahulu."); return; }

            document.getElementById('loader').style.display = 'block';
            fileProcessingQueue = Array.from(fileInput.files);

            for (const file of fileProcessingQueue) {
                await processFile(file);
            }

            document.getElementById('loader').style.display = 'none';
        }

        async function processFile(file) {
            const formData = new FormData();
            formData.append('audio_file', file);

            try {
                const response = await fetch("{{ url_for('transcribe_single_file') }}", { method: 'POST', body: formData });
                const result = await response.json();

                if (response.ok && result.status === 'success') {
                    addHistoryItem(result);
                } else {
                    alert(`Gagal memproses ${file.name}: ${result.message}`);
                }
            } catch (error) {
                alert(`Error saat mengupload ${file.name}: ${error.message}`);
            }
        }

        function addHistoryItem(result) {
            const historyList = document.getElementById('historyList');
            const noHistoryMsg = document.getElementById('noHistoryMsg');
            if(noHistoryMsg) noHistoryMsg.style.display = 'none';

            const li = document.createElement('li');
            li.innerHTML = `
                <span class="file-details"><strong>${result.original_name}</strong></span>
                <span class="file-actions">
                    <a href="${result.view_url}" target="_blank">Lihat</a> |
                    <a href="${result.download_url}">Download</a>
                </span>`;
            historyList.prepend(li); // prepend for newest on top
        }

        async function stopService() {
            // [IMPROVED] Stronger confirmation message
            const confirmation = "Anda yakin ingin MENGHENTIKAN LAYANAN dan MENGHAPUS RUNTIME COLAB?\n\nSemua proses akan berhenti total dan data yang tidak disimpan akan hilang.";
            if (!confirm(confirmation)) return;

            try {
                updateServiceStatus(false, "Mengirim permintaan untuk menghapus runtime...");
                const response = await fetch("{{ url_for('shutdown') }}", { method: 'POST' });
                const data = await response.json();
                updateServiceStatus(false, data.message + " Anda akan segera terputus.");
            } catch (error) {
                updateServiceStatus(false, "Koneksi terputus. Runtime kemungkinan sedang dihapus.");
            }
        }

        document.addEventListener('DOMContentLoaded', () => {
            const publicUrl = "{{ public_url_ngrok }}";
            const modelLoaded = "{{ model_loaded }}" === "True";
            if (modelLoaded && publicUrl && publicUrl.startsWith("http")) {
                 updateServiceStatus(true, "Layanan berjalan. URL Publik: " + publicUrl);
            } else {
                 updateServiceStatus(false, "Layanan tidak aktif. Periksa log Colab.");
            }
        });
    </script>
</body>
</html>
"""

# ------------------------------------------------------------------------------
# Flask Routes
# ------------------------------------------------------------------------------
def get_transcription_history_from_disk():
    history = []
    folder = app.config['TRANSCRIPT_FOLDER']
    try:
        files = [f for f in os.listdir(folder) if f.startswith('transcription_final_') and f.endswith('.txt')]
        sorted_files = sorted(files, key=lambda f: os.path.getmtime(os.path.join(folder, f)), reverse=True)
        for filename in sorted_files:
            original_name_part = filename[len('transcription_final_'):-len('.txt')]
            history.append({'original_name': original_name_part, 'transcript_file': filename})
    except Exception as e:
        print(f"Error membaca riwayat dari disk: {e}")
    return history

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE,
                                  initial_processed_files=get_transcription_history_from_disk(),
                                  public_url_ngrok=public_url_ngrok,
                                  model_loaded="True" if model else "False")

@app.route('/transcribe_single_file', methods=['POST'])
def transcribe_single_file():
    if 'audio_file' not in request.files:
        return jsonify({'status': 'error', 'message': "File tidak ada."}), 400
    file = request.files['audio_file']
    if file.filename == '':
        return jsonify({'status': 'error', 'message': 'Nama file kosong.'}), 400
    if file and model:
        filename_secure = secure_filename(file.filename)
        audio_path = os.path.join(app.config['UPLOAD_FOLDER'], filename_secure)
        file.save(audio_path)
        transcript_filename = transcribe_audio_web(audio_path, model, app.config['TRANSCRIPT_FOLDER'])
        return jsonify({
            'status': 'success',
            'original_name': file.filename,
            'view_url': url_for('view_transcript', filename=transcript_filename, _external=True),
            'download_url': url_for('download_transcript', filename=transcript_filename, _external=True)
        }), 200
    return jsonify({'status': 'error', 'message': 'Model tidak dimuat atau file tidak valid.'}), 500

@app.route('/download/<filename>')
def download_transcript(filename):
    response = make_response(send_from_directory(app.config['TRANSCRIPT_FOLDER'], filename, as_attachment=True))
    response.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    response.headers["Pragma"] = "no-cache"
    response.headers["Expires"] = "0"
    return response

@app.route('/view/<filename>')
def view_transcript(filename):
    return send_from_directory(app.config['TRANSCRIPT_FOLDER'], filename, mimetype='text/plain; charset=utf-8')


@app.route('/shutdown', methods=['POST'])
def shutdown():
    """
    [SIMPLIFIED] This function now disconnects ngrok and then deletes the Colab runtime.
    """
    print("🛑 Menerima permintaan shutdown & delete runtime...")

    # Disconnect ngrok tunnel
    if public_url_ngrok:
        try:
            ngrok.disconnect(public_url_ngrok)
            print("✅ Terowongan ngrok berhasil diputus.")
        except Exception as e:
            print(f"⚠️ Peringatan saat memutus ngrok: {e}")

    # Function to be called in a new thread
    def kill_runtime():
        # Wait 2 seconds to give the browser a chance to receive the response
        time.sleep(2)
        print("💥 Menghapus runtime Colab sekarang...")
        runtime.unassign() # This will terminate the entire Colab environment

    # Run the kill function in a separate thread
    threading.Thread(target=kill_runtime).start()

    # Send a final message back to the user
    return jsonify(message="Permintaan diterima. Runtime akan dihapus dalam 2 detik."), 200

# ------------------------------------------------------------------------------
# Run Flask Application and ngrok
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    if not NGROK_AUTH_TOKEN:
        print("\n🔴 APLIKASI GAGAL: Token ngrok tidak diatur.")
    elif not model:
        print("\n🔴 APLIKASI GAGAL: Model Whisper gagal dimuat.")
    else:
        # Run Flask in a separate thread
        threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, use_reloader=False)).start()
        time.sleep(3) # Wait for Flask to start

        # Start ngrok
        try:
            ngrok_tunnel = ngrok.connect(5000)
            public_url_ngrok = ngrok_tunnel.public_url
            print("====================================================================")
            print(f"✅ APLIKASI ANDA DAPAT DIAKSES DI: {public_url_ngrok}")
            print("====================================================================")
            print("ℹ️  Gunakan tombol 'Hentikan & Hapus Runtime' di web untuk mematikan.")
        except Exception as e_ngrok:
            print(f"❌ Error saat memulai ngrok: {e_ngrok}")

        # Note: No cleanup block ('finally') is needed anymore because
        # runtime.unassign() will kill this cell's execution completely.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 21.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

100%|█████████████████████████████████████| 2.88G/2.88G [00:43<00:00, 70.4MiB/s]


✅ Whisper model successfully loaded.
🚀 Starting Flask server...
⏳ Waiting for Flask server to be ready...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


🔌 Creating ngrok tunnel to port 5000...
✅ Your application is accessible at: https://18d7-35-198-231-216.ngrok-free.app
ℹ️  Server is running. Use the 'Stop Service' button on the web or stop this Colab cell to shut down.


...

✅ Transcription finished! Output saved to 'transcripts/transcription_final_AUD-20250527-WA0004.txt'. Detected language by Whisper: id


INFO:werkzeug:127.0.0.1 - - [27/May/2025 07:20:42] "GET /download/transcription_final_AUD-20250527-WA0004.txt HTTP/1.1" 200 -


DEBUG: Download request received for filename: 'transcription_final_AUD-20250527-WA0004.txt'
DEBUG: Attempting to send file from directory: '/content/transcripts'
DEBUG: Constructed full file path for download: 'transcripts/transcription_final_AUD-20250527-WA0004.txt'
DEBUG: File 'transcription_final_AUD-20250527-WA0004.txt' located. Preparing to send.
DEBUG: Sending file 'transcription_final_AUD-20250527-WA0004.txt' with cache-control headers.
🛑 Received shutdown request from client...
🔌 Disconnecting ngrok tunnel...


INFO:werkzeug:127.0.0.1 - - [27/May/2025 07:30:21] "POST /shutdown HTTP/1.1" 200 -


✅ Ngrok tunnel disconnected successfully.
🔪 Killing ngrok process...
✅ Ngrok process killed successfully.
⚠️ Cannot find Werkzeug shutdown function. Server might need manual stop.
ℹ️ Server was stopped.
🧹 Performing cleanup as server is marked to stop or failed to start...
🧹 Starting final cleanup procedure (main `finally` block)...
   Shutting down ngrok (if active)...
   No active ngrok tunnels found to disconnect.
   Ngrok process kill attempted successfully (or was not running).
   Flask thread is still alive. Waiting for it to stop...
